In [2]:
import requests
from bs4 import BeautifulSoup

# Function to check if the URL uses HTTPS for secure communication
def check_https(url):
    if url.startswith('https'):
        print("HTTPS is enabled.")
    else:
        print("HTTPS is not enabled. Consider using HTTPS for better security.")

# Function to find login or signup links in the webpage
def find_login_signup_links(base_url):
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    login_links = []
    signup_links = []

    # Loop through all anchor tags to check for login or signup-related links
    for link in soup.find_all('a', href=True):
        href = link['href']
        if 'login' in href.lower():
            login_links.append(href)
        if 'signup' in href.lower() or 'register' in href.lower():
            signup_links.append(href)

    return login_links, signup_links

# Function to analyze login form structure
def analyze_login_form(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all input fields in the login form
    inputs = soup.find_all('input')
    for input_field in inputs:
        print(f"Input name: {input_field.get('name')}, type: {input_field.get('type')}")

    # Check for CAPTCHA protection
    captcha = soup.find('div', {'class': 'g-recaptcha'})
    if captcha:
        print("CAPTCHA is present, indicating added security.")
    else:
        print("CAPTCHA is not present, which might make brute force attacks easier.")

# Function to inspect cookies and authentication tokens
def inspect_cookies_and_tokens(session, login_url, username, password):
    login_payload = {
        'username': username,
        'password': password
    }
    login_response = session.post(login_url, data=login_payload)

    # Inspect cookies for security attributes
    for cookie in session.cookies:
        print(f"Cookie: {cookie.name}, Secure: {cookie.secure}, HttpOnly: {cookie.has_nonstandard_attr('HttpOnly')}")

    # Check if JWT tokens are used in authentication
    auth_header = login_response.headers.get('Authorization')
    if auth_header and 'Bearer' in auth_header:
        print("JWT token is used for authentication.")
    else:
        print("JWT token is not used. Authentication method needs review.")

# Function to analyze session management security
def analyze_session_management(session):
    # Check the length and randomness of the session ID
    session_id = session.cookies.get('sessionid')
    if session_id and len(session_id) > 20:
        print("Session ID is long and likely random, improving security.")
    else:
        print("Session ID is short or not random, which might be a security risk.")

    # Check session expiration settings
    session_cookie = session.cookies.get('sessionid', domain='bekushal.com')
    if session_cookie:
        session_expiry = session_cookie.expires
        if session_expiry:
            print(f"Session expires at: {session_expiry}, which is a good security practice.")
        else:
            print("Session expiration is not set, which might indicate a security risk.")
    else:
        print("Session cookie not found, indicating possible issues with session management.")

# Function to perform authorization checks
def authorization_checks(session, protected_url):
    protected_response = session.get(protected_url)

    # Check if access is properly restricted
    if protected_response.status_code == 403:
        print("Access is correctly restricted based on user roles.")
    elif protected_response.status_code == 200:
        print("Access is granted. Ensure that authorization checks are properly implemented.")
    else:
        print("Unexpected response status code, review access control mechanisms.")

# Function to review important HTTP security headers
def review_http_headers(response):
    headers = response.headers

    security_headers = [
        'Strict-Transport-Security',
        'Content-Security-Policy',
        'X-Content-Type-Options',
        'X-Frame-Options'
    ]

    # Check if essential security headers are present
    for header in security_headers:
        if header in headers:
            print(f"{header} is present, which enhances security.")
        else:
            print(f"{header} is missing, consider adding it for better security.")

# Main function to run all security checks
def main():
    base_url = 'https://amazon.in'  # Change this to the target website
    session = requests.Session()

    # Step 1: Check if HTTPS is enabled
    check_https(base_url)

    # Step 2: Identify login and signup links
    login_links, signup_links = find_login_signup_links(base_url)
    print(f"Login links found: {login_links}")
    print(f"Signup links found: {signup_links}")

    # Step 3: Analyze login form if login page is found
    if login_links:
        analyze_login_form(base_url + login_links[0])

    # Step 4: Inspect cookies and authentication tokens
    if login_links:
        username = 'your_username'  # Replace with actual username
        password = 'your_password'  # Replace with actual password
        inspect_cookies_and_tokens(session, base_url + login_links[0], username, password)

    # Step 5: Check session management security
    analyze_session_management(session)

    # Step 6: Perform authorization checks (Ensure correct access control)
    protected_url = base_url + '/protected'  # Update with actual protected resource URL
    authorization_checks(session, protected_url)

    # Step 7: Review HTTP security headers
    response = session.get(base_url)
    review_http_headers(response)

if __name__ == "__main__":
    main()


HTTPS is enabled.
Login links found: []
Signup links found: []
Session ID is short or not random, which might be a security risk.
Session cookie not found, indicating possible issues with session management.
Unexpected response status code, review access control mechanisms.
Strict-Transport-Security is present, which enhances security.
Content-Security-Policy is present, which enhances security.
X-Content-Type-Options is present, which enhances security.
X-Frame-Options is present, which enhances security.


In [12]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://sitare.org'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Extract emails
emails = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', soup.text)
print('Emails:', emails)

# Extract phone numbers
phone_numbers = re.findall(r'\b\d{10}\b', soup.text)
print('Phone Numbers:', phone_numbers)

# Extract privacy policy URL
privacy_policy = soup.find('a', href=True, text=re.compile(r'Privacy Policy', re.I))
if privacy_policy:
    privacy_policy_url = privacy_policy['href']
    print(f'Privacy Policy URL: {privacy_policy_url}')


Emails: []
Phone Numbers: []


<ipython-input-12-c5d4151b2cff>:18: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  privacy_policy = soup.find('a', href=True, text=re.compile(r'Privacy Policy', re.I))
